In [1]:
import torch
from IPython import display
from d2l import torch as d2l

batch_siz=256
train_iter,test_iter =d2l.load_data_fashion_mnist(batch_siz)

In [2]:
num_inputs=784
num_outputs=10

In [3]:
W=torch.normal(0,0.01,size=(num_inputs,num_outputs),requires_grad=True)
b=torch.zeros(num_outputs,requires_grad=True)

In [4]:
def softmax(X):
    X_exp=torch.exp(X)
    partition=X_exp.sum(1,keepdim=True)
    return X_exp/partition

In [1]:
 #实现 softmax 回归
def net(X):
    #-1 表示根据X自己推断  这是输入X【256，1，28，28] reshape之后是[256,784】
    return softmax(torch.matmul(X.reshape((-1,W.shape[0])),W)+b)

In [2]:
def cross_entropy(y_hat,y):
    return -torch.log(y_hat[range(len(y_hat)),y])

In [3]:
def accuracy(y_hat,y):
    if len(y_hat.shape)>1 and y_hat.shape[1]>1:
        y_hat=y_hat.argmx(axis=1)
    cmp=y_hat.type(y.dtpye)==y
    return float(cmp.type(y.dtpye).sum())


In [4]:
def evaluate_accuracy(net,dat_iter):
    if isinstance(net,torch.nn.Module):
        net.eval()
    matric=Accumulator(2)
    for X, y in dat_iter:
        matric.add(accuracy(net(X),y),y.numel())
    return matric[0]/matric[1]

In [5]:
class Accumulator:
    def __init__(self,n):
        self.data=[0.0]*n
    def add(self,*args):
        self.data=[a+float(b) for a,b in zip (self.data,args)]
    def reset(self):
        self.data=[0.0]*len(self.data)
    def __getitem__(self, item):
        return self.data[item]

In [8]:
def train_epoch_ch3(net,train_iter,loss,updater):
    if isinstance(net,torch.nn.Module):
        net.train()
    metric=Accumulator(3)
    for X,y in train_iter:
        y_hat=net(X)
        l=loss(y_hat,y)
        if isinstance(updater,torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            updater.step()
            metric.add(float(1)*len(y),accuracy(y_hat,y),
                       y.size().numel())
        else:
            l.sum().backward()
            updater(X.shape[0])
            metric.add(float(1)*len(y),accuracy(y_hat,y),
                       y.size().numel())
    return metric[0]/metric[2],metric[1]/metric[2]